In [ ]:
import os
import pandas as pd

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#os.chdir("/content/drive/MyDrive/Harvey Associate Initiative/")
os.chdir("/Users/rerobbins/projects/work/harvey_training_initiative")

In [ ]:
offices = pd.read_excel("2024-12-28 -- Lookup Tables.xlsx", sheet_name="Offices")
titles = pd.read_excel("2024-12-28 -- Lookup Tables.xlsx", sheet_name="Titles")

In [ ]:
offices

In [ ]:
titles

In [ ]:
# Load the data
firm = pd.read_excel("2025-01-10 -- Firm Directory From People Finder.xlsx")

# Rename the first column to 'Email' and standardize its format
firm = firm.rename(columns={firm.columns[0]: "Email"})
firm["Email"] = firm["Email"].str.strip().str.lower()

# Filter for primary office and drop rows with missing 'Email' or 'Department'
firm = firm[firm['Office Type'] == "Primary Office"]
firm = firm.dropna(subset=['Email', 'Department'])

# Split 'Department' into 'Department' and 'Group', stripping whitespace
firm[['Department', 'Group']] = firm['Department'].str.split(' - ', n=1, expand=True)
firm['Department'] = firm['Department'].str.strip()
firm['Group'] = firm['Group'].str.strip()

# Extract the first part of 'Title' into 'Title Stem'
firm['Title Stem'] = firm['Title'].str.split('<br/>').str[0].str.strip()

firm = firm.merge(offices, on="Office", how='left')
firm = firm.merge(titles, on="Title Stem", how='left')

columns = ['Email', 'Name', 'Title', 'Title Stem', 'Title Group', 'Attorney',
           'Department', 'Group', 'Office', 'Region']

firm = firm[columns]

# Display the cleaned DataFrame
firm

In [ ]:
# The firm data pulled from People Finder on Ourspace doesn't fully reflect
# Group and Department assignments for U.S. New Associates until the start of
# the calendar year following their start dates, so we patch this gap.

#new_associates_mapping = pd.read_excel("2024-12-15 -- US New Associates.xlsx")
#new_associates_mapping

In [ ]:
#firm = firm.merge(
#    new_associates_mapping[['Email', 'Department', 'Group']],
#    on='Email',
#    how='left',
#    suffixes=('', '_new')  # so we get Department_new, Group_new
#)

# Update department and group only where Title Stem == "New Associate"
#mask = firm['Title Stem'] == 'New Associate'
#firm.loc[mask, 'Department'] = firm.loc[mask, 'Department_new']
#firm.loc[mask, 'Group']      = firm.loc[mask, 'Group_new']

# Drop the extra columns
#firm.drop(columns=['Department_new', 'Group_new'], inplace=True)


In [ ]:
firm[(firm['Region'] == 'U.S.') & (firm['Title Stem'] == 'Associate')].Group.value_counts()

In [ ]:
firm[(firm['Region'] == 'U.S.') & (firm['Title Stem'] == 'Associate')].Department.value_counts()

In [ ]:
firm.Region.value_counts()

In [ ]:
firm.to_excel("firm_directory.xlsx", index=False)